#### Package inputs

In [8]:
import os
import pandas as pd
import geopandas as gpd
import re
import numpy as np
from shapely.geometry import Point, LineString, shape, Polygon, MultiPoint

#### Specify parameters

##### Specify input paths

In [9]:
# set working directory to root
os.chdir('/mnt/c/Users/Sean.Kearney/git_repos/cper_hls_veg_models')

# transects shapefile
transects_f = "/mnt/t/3-GIS/CPER/Layers/Plots_Transects/CPER_VOR_Transects_all_2020_Oct14.shp"

##### Specify cleaning parameters

In [10]:
# create transect pasture renaming dictionary
trans_name_dict = {
    'NH': '10S',
    '7se': '7SE',
    '31w': '31W',
    '26w': '26W',
    '25nw': '25NW',
    '17s': '17S',
    '18s': '18S',
    '19n': '19N',
    '21n': '21N',
    '22SW': '22W'
}

##### Read in datasets

In [24]:
# read in shapefiles
gdf_transects = gpd.read_file(transects_f)

##### Clean up and prepare transects and plots

In [21]:
# rename transect pasture names to match pasture shapefile
gdf_transects['PastureNm'] = gdf_transects['PastureNm'].apply(
    lambda x: trans_name_dict[x] if x in trans_name_dict.keys() else x)

# remove prefixes for transect names
gdf_transects['Transect'] = gdf_transects['Transect'].apply(
    lambda x: re.sub("P", "", re.sub("T", "", str(x))))

# create unique transect id's
gdf_transects['Id'] = gdf_transects.apply(
    lambda x: x['PastureNm'] + "_" + x['Plot_Num'] + "_" + str(x['Transect']),
    axis=1)

# create unique plot id's
gdf_transects['Id_plot'] = gdf_transects.apply(lambda x: x['PastureNm'] + "_" + x['Plot_Num'], axis=1)

# create transect geometry from coordinates
#gdf_transects['geometry'] = gdf_transects['geometry'].apply(lambda x: x.coords[0])

# create plot-scale geodataframe using convex hull and buffer by 15 m
gdf_plots = gdf_transects.groupby(['PastureNm', 'Id_plot'])['geometry'].apply(lambda x: MultiPoint(x.tolist()).convex_hull)
gdf_plots = gpd.GeoDataFrame(gdf_plots, geometry='geometry', crs=gdf_transects.crs).buffer(15)
gdf_plots.name = 'geometry'
gdf_plots = gdf_plots.reset_index()

# convert transect-scale geodataframe to polygon using convex hull and buffer by 15 m
gdf_transects = gdf_transects.groupby(['PastureNm', 'Id'])['geometry'].apply(lambda x: MultiPoint(x.tolist()).convex_hull)
gdf_transects = gpd.GeoDataFrame(gdf_transects, geometry='geometry', crs=gdf_plots.crs).buffer(15)
gdf_transects.name = 'geometry'
gdf_transects = gdf_transects.reset_index()

##### Save transects and plots to disk

In [27]:
gdf_transects.to_file("./data/ground/boundaries/CPER_VOR_Transects_all_2020_Oct14_clean.shp")
gdf_plots.to_file("./data/ground/boundaries/CPER_plots_from_vor_transects.shp")

In [26]:
gdf_plots

,PastureNm,Id_plot,geometry
0,10S,10S_P1,"POLYGON ((519555.037 4521231.115, 519553.566 4..."
1,10S,10S_P2,"POLYGON ((519544.297 4520979.580, 519542.826 4..."
2,10S,10S_P3,"POLYGON ((519289.993 4520977.318, 519288.522 4..."
3,10S,10S_P4,"POLYGON ((519297.653 4521230.160, 519296.182 4..."
4,15E,15E_P1,"POLYGON ((520181.470 4520621.453, 520180.000 4..."
...,...,...,...
127,8E,8E_P4,"POLYGON ((526590.786 4522346.063, 526589.316 4..."
128,CN,CN_P1,"POLYGON ((519747.373 4516902.969, 519745.902 4..."
129,CN,CN_P2,"POLYGON ((519578.393 4517098.704, 519576.922 4..."
130,CN,CN_P3,"POLYGON ((519345.172 4516991.944, 519343.701 4..."
